In [375]:
import sqlite3
import pandas as pd

In [376]:
feature_id = 18
feature_db_name = '/home/ubuntu/UPS2_DIA_180907/UPS2_DIA_180907-features-{}-{}.sqlite'.format(feature_id, feature_id)
mz_tolerance_ppm = 2

In [377]:
db_conn = sqlite3.connect(feature_db_name)
features_df = pd.read_sql_query("select * from feature_list where feature_id == {} order by feature_id ASC;".format(feature_id), db_conn)
db_conn.close()

In [378]:
features_df

,feature_id,charge_state,monoisotopic_mass,centroid_scan,centroid_rt,centroid_mz,start_rt,end_rt,scan_lower,scan_upper,summed_intensity,isotope_count
0,18,3,1414.705462,507.692868,390.43289,472.80709,387.415429,392.738298,499,518,522144,4


In [379]:
db_conn = sqlite3.connect(feature_db_name)
ms2_peaks_df = pd.read_sql_query("select * from ms2_peaks where feature_id={}".format(feature_id), db_conn)
db_conn.close()

In [380]:
ms1_centroid_mz = features_df.iloc[0].centroid_mz.astype(float)
ms1_centroid_scan = features_df.iloc[0].centroid_scan.astype(float)

In [381]:
ms2_peaks_df.head()

,feature_id,peak_id,centroid_mz,composite_mzs_min,composite_mzs_max,centroid_scan,intensity,cofi_scan,cofi_rt
0,18,1,472.575,472556,472596,514,1346847,514.523651,390.379934
1,18,2,472.909,472889,472929,514,1046961,514.756696,390.381650
2,18,3,473.244,473225,473265,514,491294,514.827175,390.394725
3,18,4,473.577,473558,473598,514,161460,514.626867,390.308027
4,18,5,336.920,336909,336933,503,166120,503.126824,388.891867


In [382]:
db_conn = sqlite3.connect(feature_db_name)
feature_isotopes_df = pd.read_sql_query("select * from feature_isotopes where feature_id == {} order by feature_id ASC;".format(feature_id), db_conn)
db_conn.close()

In [383]:
feature_isotopes_df

,feature_id,feature_region_peak_id,centroid_scan,centroid_rt,centroid_mz,peak_summed_intensity
0,18,1,507.385797,390.527168,472.575875,254968
1,18,2,507.661874,390.549442,472.910193,191754
2,18,3,508.606541,389.852057,473.244341,56875
3,18,4,509.432846,389.712975,473.578808,18547


In [384]:
base_peak_centroid_mz = feature_isotopes_df.iloc[feature_isotopes_df.peak_summed_intensity.idxmax()].centroid_mz

In [385]:
base_peak_centroid_mz

472.5758752551706

In [386]:
ms2_peaks_df['mz_delta'] = abs(ms2_peaks_df.centroid_mz - base_peak_centroid_mz)

In [387]:
indexes_to_drop = ms2_peaks_df.mz_delta > (mz_tolerance_ppm * 10**-6 * base_peak_centroid_mz)

In [388]:
ms2_peaks_df.drop(ms2_peaks_df.index[indexes_to_drop], inplace=True)

In [389]:
ms2_peaks_df

,feature_id,peak_id,centroid_mz,composite_mzs_min,composite_mzs_max,centroid_scan,intensity,cofi_scan,cofi_rt,mz_delta
0,18,1,472.575,472556,472596,514,1346847,514.523651,390.379934,0.000875
